<a href="https://colab.research.google.com/github/MaliheDahmardeh/state-house-price-analysis-SM/blob/main/state-house-price-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil import parser
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
!ls

sample_data  State_house_prices_ts.csv


In [3]:
df = pd.read_csv('State_house_prices_ts.csv', sep =',',parse_dates=['Date'])

In [4]:
display(df)
display(type(df))
display(df.dtypes)

,Date,RegionName,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
0,2001-02-28,Alabama,70100.0,58700.0,95900.0,178700.0,248100.0,92500.0
1,2001-02-28,Alaska,97300.0,126400.0,177300.0,219800.0,282500.0,183300.0
2,2001-02-28,Arizona,75800.0,110200.0,126600.0,168100.0,250500.0,136900.0
3,2001-02-28,Arkansas,65100.0,64500.0,90300.0,156700.0,245300.0,74400.0
4,2001-02-28,California,146900.0,182700.0,218400.0,293700.0,405100.0,239900.0
...,...,...,...,...,...,...,...,...
13207,1999-12-31,Virginia,80800.0,84400.0,114300.0,191000.0,266500.0,125100.0
13208,1999-12-31,Washington,115300.0,131300.0,164400.0,218700.0,254000.0,165600.0
13209,1999-12-31,WestVirginia,NaN,NaN,60900.0,80200.0,90000.0,48100.0
13210,1999-12-31,Wisconsin,80800.0,89800.0,114400.0,143400.0,165400.0,113700.0


pandas.core.frame.DataFrame

Date                           datetime64[ns]
RegionName                             object
Price_1bedroom                        float64
Price_2bedroom                        float64
Price_3bedroom                        float64
Price_4bedroom                        float64
Price_5BedroomOrMore                  float64
Price_SingleFamilyResidence           float64
dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13212 entries, 0 to 13211
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Date                         13212 non-null  datetime64[ns]
 1   RegionName                   13212 non-null  object        
 2   Price_1bedroom               10605 non-null  float64       
 3   Price_2bedroom               11745 non-null  float64       
 4   Price_3bedroom               12787 non-null  float64       
 5   Price_4bedroom               12359 non-null  float64       
 6   Price_5BedroomOrMore         11814 non-null  float64       
 7   Price_SingleFamilyResidence  12438 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 825.9+ KB


In [6]:
df.duplicated().sum()  

0

In [7]:
df.isnull().sum()

Date                              0
RegionName                        0
Price_1bedroom                 2607
Price_2bedroom                 1467
Price_3bedroom                  425
Price_4bedroom                  853
Price_5BedroomOrMore           1398
Price_SingleFamilyResidence     774
dtype: int64

In [ ]:
number=df.groupby('RegionName').count()[['Price_1bedroom','Price_2bedroom','Price_3bedroom','Price_3bedroom','Price_5BedroomOrMore','Price_SingleFamilyResidence']].reset_index()
#display(type(number))
number

In [29]:
Region = df['RegionName'].unique().tolist()
Region.sort()

In [33]:
def calc_count(variable,value):
  for variable in Region :
    a = number.loc[(number["RegionName"]== variable)]
    b = a[value]
    return b
m= calc_count('Alabma','Price_1bedroom')
m

0    261
Name: Price_1bedroom, dtype: int64

In [28]:
house_type = ['Price_1bedroom','Price_2bedroom','Price_3bedroom','Price_3bedroom','Price_5BedroomOrMore','Price_SingleFamilyResidence']
#display(house_type)
#display(type(house_type))

In [36]:
# Median --> Returns the middle data point in a data set 
def calc_median(variable,value):
  n = calc_count(variable,value)
  if   n%2 == 1: # n is odd 
        df = np.sort(variable)
        df_var = df[(n)//2]
  elif n%2 == 0: # n is even 
        df = np.sort(variable)
        df_var = (df[n//2-1] + df[(n//2)])/2 # be careful with Python indexes, starting from 0 
  return df_var

T5 = calc_median('Alabma','Price_1bedroom')
T5

ValueError: ignored

In [ ]:
df.describe()

,Price_1bedroom,Price_2bedroom,Price_3bedroom,Price_4bedroom,Price_5BedroomOrMore,Price_SingleFamilyResidence
count,10605.000000,11745.000000,12787.000000,12359.000000,1.181400e+04,12438.000000
mean,117060.424328,135168.803746,167062.798154,243829.670685,3.237335e+05,174154.381733
std,61056.036728,72267.645405,80212.505902,106216.755129,1.702917e+05,92243.367198
min,30900.000000,32800.000000,49600.000000,64700.000000,6.860000e+04,37900.000000
25%,74600.000000,86700.000000,116400.000000,174900.000000,2.179000e+05,115000.000000
50%,100400.000000,115400.000000,141200.000000,218000.000000,2.880000e+05,147300.000000
75%,142300.000000,166800.000000,204400.000000,281000.000000,3.653000e+05,211775.000000
max,390200.000000,542400.000000,639700.000000,850400.000000,1.497000e+06,737500.000000


In [ ]:
print('columns:', df.columns)
print('')
print('indexes:', df.index)
print('')
print('values:', df.values)

columns: Index(['Date', 'RegionName', 'Price_1bedroom', 'Price_2bedroom',
       'Price_3bedroom', 'Price_4bedroom', 'Price_5BedroomOrMore',
       'Price_SingleFamilyResidence'],
      dtype='object')

indexes: RangeIndex(start=0, stop=13212, step=1)

values: [[Timestamp('2001-02-28 00:00:00') 'Alabama' 70100.0 ... 178700.0
  248100.0 92500.0]
 [Timestamp('2001-02-28 00:00:00') 'Alaska' 97300.0 ... 219800.0 282500.0
  183300.0]
 [Timestamp('2001-02-28 00:00:00') 'Arizona' 75800.0 ... 168100.0
  250500.0 136900.0]
 ...
 [Timestamp('1999-12-31 00:00:00') 'WestVirginia' nan ... 80200.0 90000.0
  48100.0]
 [Timestamp('1999-12-31 00:00:00') 'Wisconsin' 80800.0 ... 143400.0
  165400.0 113700.0]
 [Timestamp('1999-12-31 00:00:00') 'Wyoming' nan ... 116600.0 nan
  104100.0]]


#Data manipulation